In [9]:
import pandas as pd

import os

import numpy as np

import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras

from IPython.display import clear_output

In [10]:
last_csv = int(np.load('data/get_comment_threads_output/last_csv.npy'))

In [11]:
df_videos = pd.read_csv('data/get_videos_details/videos_details_computed.csv')
comment_count = dict([(item[0],0) for item in df_videos.values])

In [12]:
model = tf.keras.models.load_model('sentiment_analysis_model.h5',custom_objects={'KerasLayer':hub.KerasLayer})

In [13]:
limit = 5
def reduce_dataset(df,dict_videos):
    arr_reduced = []
    for row in df.values:
        if dict_videos[row[0]] < limit:
            arr_reduced.append(row)
            dict_videos[row[0]] += 1
    df_reduced = pd.DataFrame(arr_reduced, columns = df.columns)
    return df_reduced

In [14]:
def compute_sentiment(df):
    arr_reduced=[]
    total_predictions = []
    dataset = tf.data.Dataset.from_tensor_slices(df['text_original'])
    dataset = dataset.batch(32).prefetch(1)
    idx=0
    for item in dataset:
        predictions = model.predict(item)
        for pred in predictions:
            print('Comment: {}/{}'.format(idx,len(df.values)))
            clear_output(wait=True)
            idx+=1
            total_predictions.append(1 if pred>=0.5 else 0)
    df_computed = df.copy()
    df_computed = df_computed.assign(sentiment=pd.Series(total_predictions))
    return df_computed

In [15]:
for i in range(last_csv+1):
    print('Computing {} csv'.format(i))
    #questo l'ho messo perchè i primi due li ho già fatti a mano
    #if i not in [0,1]:
    df_comments = pd.read_csv('data/get_comment_threads_output/df_comment_thread_{}.csv'.format(i))
    df_red = reduce_dataset(df_comments,comment_count)
    df_sentiment = compute_sentiment(df_red)
    df_sentiment_reduced = df_sentiment[df_sentiment.columns[[0,2,3,6]]]
    if i == 0:
        df_sentiment_complete = df_sentiment_reduced.copy()
    else:
        df_sentiment_complete = df_sentiment_complete.append(df_sentiment_reduced)
        
df_sentiment_complete.to_csv('data/get_comment_threads_output/comments_thread_sentiment_reduced_{}.csv'.format(limit), index = False)

Comment: 899/900


In [16]:
df_sentiment_complete

,video_id,top_level_comment_id,author_display_name,sentiment
0,WyUyJKm-MiY,UgxmeuEGKaTnjr18AVZ4AaABAg,Imaan Jaffri,0
1,WyUyJKm-MiY,UgzlwPwyC3XlmzgzBr54AaABAg,Karlee Nash,0
2,WyUyJKm-MiY,UgynycPfXmKJePLzAEp4AaABAg,Bernice Ayer,1
3,WyUyJKm-MiY,UgwamwomyWMY8bpiPjp4AaABAg,Molly Seymour,1
4,WyUyJKm-MiY,Ugx7NWDtnzTkeACe5yJ4AaABAg,Charbonelle Weese,0
...,...,...,...,...
895,8kElrRAa1ws,UgwFBotqMk8PHYtjGlJ4AaABAg,Liana Soares,1
896,8kElrRAa1ws,UgwSdanGKGsGd4DUmIl4AaABAg,Bree e,0
897,8kElrRAa1ws,Ugy-wtIUI069bRoN9K14AaABAg,Tonya Wykle,1
898,8kElrRAa1ws,UgwtkQootxJ6rfJzvPh4AaABAg,Lesly,0
